In [0]:
!pip uninstall pennylane
!pip install pennylane-sf

In [0]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer, AdamOptimizer
from pennylane.templates.layers import *
import math

In [0]:

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline


plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.sans-serif'] = ['Computer Modern Roman']
plt.style.use('default')

In [0]:
def fidelity(A, B):
    #if A.dims != B.dims:
    #    raise TypeError('Density matrices do not have same dimensions.')
        
    return float(np.sqrt(np.real((A * (B * A)))).trace())
  
  
def wigner(rho):

    import copy
    l = 5.0
    cutoff = rho.shape[0]

    x = np.linspace(-l, l, 100)
    p = np.linspace(-l, l, 100)

    Q, P = np.meshgrid(x, p)
    A = (Q + P * 1.0j) / (2 * np.sqrt(2 / 2))

    Wlist = np.array([np.zeros(np.shape(A), dtype=complex) for k in range(cutoff)])
    Wlist[0] = np.exp(-2.0 * np.abs(A) ** 2) / np.pi
    W = np.real(rho[0, 0]) * np.real(Wlist[0])

    for n in range(1, cutoff):
        Wlist[n] = (2.0 * A * Wlist[n - 1]) / np.sqrt(n)
        W += 2 * np.real(rho[0, n] * Wlist[n])

    for m in range(1, cutoff):
        temp = copy.copy(Wlist[m])
        Wlist[m] = (2 * np.conj(A) * temp - np.sqrt(m)
                    * Wlist[m - 1]) / np.sqrt(m)
        W += np.real(rho[m, m] * Wlist[m])

        for n in range(m + 1, cutoff):
            temp2 = (2 * A * Wlist[n - 1] - np.sqrt(m) * temp) / np.sqrt(n)
            temp = copy.copy(Wlist[n])
            Wlist[n] = temp2
            W += 2 * np.real(rho[m, n] * Wlist[n])

    return Q, P, W / 2

In [0]:
dev = qml.device('strawberryfields.fock', wires=3, cutoff_dim=10)

In [0]:
THETA1, PHI1, VARPHI1, R, PHIR, THETA2, PHI2, VARPHI2, A, PHIA, K = (i for i in range(11))

In [0]:
MODULUS = 1

def real(phi):
    # generates states that are unit distance from vacuum state
    qml.Displacement(MODULUS, phi, wires=1)

In [0]:
def layer(thetas, phis, rphis, S, Dr, Dp, K, q):
    
    N = len(q)
    
    def interferometer(theta, phi, rphi, q):
      '''
        args :
          theta : list of N(N-1)/2 params
          phi : list of N(N-1)/2 params
          rphi : list of N-1 params
      '''
      N = len(q)
      if N == 1:
        qml.Rotation(rphi[0], wires=q[0])
        return
      
      n = 0
      for l in range(N):
        for k, (q1, q2) in enumerate(zip(q[:-1], q[1:])):
          if (l+k)%2 != 1:
            qml.Beamsplitter(theta[n], phi[n], wires=(q1,q2))
            n+=1
            
      for i in range(max(1, N-1)):
        qml.Rotation(rphi[i], wires=q[i])
        
      return
    
    interferometer(thetas[0], phis[0], rphis[0], q)
    for i in range(N):
      qml.Squeezing(S[i], 0.0, wires=q[i])
    interferometer(thetas[1], phis[1], rphis[1], q)
    
    for i in range(N):
      qml.Displacement(Dr[i], Dp[i], wires=q[i])
      qml.Kerr(K[i], wires[i])
    
      

def generator(gw, z=1, label = None):

    # z is random real number
    q = [1,2]
    qml.Displacement(z, 0.0, wires = 2)
    #layer(g['thetas'], g['phis'], g['rphis'], g['S'], g['Dr'], g['Dp'], g['K'], q)
    
    
    idx = 0
    g = []
    for i in range(11):
      k = gsize[i][0]*gsize[i][1]
      g.append(np.reshape(gw[idx:idx + k],gsize[i]))
      idx += k
      
    CVNeuralNetLayers(g[THETA1], g[PHI1], g[VARPHI1], g[R], g[PHIR], g[THETA2], g[PHI2], g[VARPHI2], g[A], g[PHIA], g[K], wires=q)
    
    
    
def discriminator(dw, label = None):
    q = [0,1]
    #layer(d['thetas'], d['phis'], d['rphis'], d['S'], d['Dr'], d['Dp'], d['K'], q)
    idx = 0
    d = []
    for i in range(11):
      k = dsize[i][0]*dsize[i][1]
      d.append(np.reshape(dw[idx:idx + k],gsize[i]))
      idx += k
    CVNeuralNetLayers(d[THETA1], d[PHI1], d[VARPHI1], d[R], d[PHIR], d[THETA2], d[PHI2], d[VARPHI2], d[A], d[PHIA], d[K], wires=q)
    
    

In [0]:
@qml.qnode(dev)
def real_disc_circuit(phi, disc_weights):
    real(phi)
    discriminator(disc_weights)
    return qml.expval.X(0)

@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval.X(0)


###Evaluation Circuits


In [34]:
dev2 = qml.device('strawberryfields.fock', wires=3, cutoff_dim=10)
dev2
@qml.qnode(dev2)
def trained_gen(gen_weights):
    generator(gen_weights)
    return qml.expval.X(0)
  
'''trained_gen(gen_weights)
rho_gen = dev2.state.reduced_dm(1)
rho_gen.shape'''

'trained_gen(gen_weights)\nrho_gen = dev2.state.reduced_dm(1)\nrho_gen.shape'

In [35]:
dev3 = qml.device('strawberryfields.fock', wires=3, cutoff_dim=10)
@qml.qnode(dev3, interface='torch')
def real_distr():
  real(phi)
  return qml.expval.X(0)

'''real_distr()
rho_real = dev3.state.reduced_dm(1)'''

'real_distr()\nrho_real = dev3.state.reduced_dm(1)'

###Probability Measures

In [0]:

def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, disc_weights)
    # convert to probability
    prob_real_true = 1 / (1+math.e**(-true_disc_output))
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = 1 / (1+math.e**(-fake_disc_output))
    return prob_fake_true # generator wants to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    #cost = math.log(prob_real_true(disc_weights)) + math.log(1-prob_fake_true(gen_weights, disc_weights))
    return -cost

def gen_cost(gen_weights):
    #cost = math.log(prob_fake_true(gen_weights, disc_weights))
    return -prob_fake_true(gen_weights, disc_weights)

In [59]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
#gen_weights = np.array([np.pi] + [0] * 8) + np.random.normal(scale=eps, size=[9])
#disc_weights = np.random.normal(size=[9])

GL = 1
GM = 2
GK = 1 #(GM * (GM-1))/2

DL = 1
DM = 2
DK = 1 #(DM * (DM-1))/2

THETA1, PHI1, VARPHI1, R, PHIR, THETA2, PHI2, VARPHI2, A, PHIA, K = (i for i in range(11))

gsize = [(GL,GK) , (GL,GK) , (GL,GM) , (GL,GM) , (GL,GM) , (GL,GK) , (GL,GK) , (GL,GM) , (GL,GM) , (GL,GM) , (GL,GM)]
dsize = gsize
tg = 0
for i in gsize: tg += i[0]*i[1]
td = tg
'''gen_weights = [
    np.random.rand(GL, GK),
    np.random.rand(GL, GK),
    np.random.rand(GL, GM),
    np.random.rand(GL, GM),
    np.random.rand(GL, GM),
    np.random.rand(GL, GK),
    np.random.rand(GL, GK),
    np.random.rand(GL, GM),
    np.random.rand(GL, GM),
    np.random.rand(GL, GM),
    np.random.rand(GL, GM)
]'''
gen_weights = np.random.rand(tg)
'''disc_weights = [
    np.random.rand(DL, DK),
    np.random.rand(DL, DK),
    np.random.rand(DL, DM),
    np.random.rand(DL, DM),
    np.random.rand(DL, DM),
    np.random.rand(DL, DK),
    np.random.rand(DL, DK),
    np.random.rand(DL, DM),
    np.random.rand(DL, DM),
    np.random.rand(DL, DM),
    np.random.rand(DL, DM)])'''
disc_weights = np.random.rand(td)
type(gen_weights)

numpy.ndarray

###Training

In [0]:
opt = AdamOptimizer(0.1)

In [0]:
f = []
best_fl = [0]
best_dw = disc_weights
best_gw = gen_weights
best_f = 0

for adv in range(30):
    print("Iteration : ", adv)
    print("discriminator training ...")
    for it in range(30):
        disc_weights = opt.step(disc_cost, disc_weights) 
        #cost = disc_cost(disc_weights)
        #if it % 15 == 0:
        #    print("Step {}: cost = {}".format(it+1, cost))
    print("Discriminator training over .. probability real true : {} \n probability fake true : {}".format(prob_real_true(disc_weights), prob_fake_true(gen_weights, disc_weights)))
    print("generator training ...")
    for it in range(100):
        gen_weights = opt.step(gen_cost, gen_weights)
        #cost = -gen_cost(gen_weights)
        #if it % 25 == 0:
        #    print("Step {}: cost = {}".format(it, cost))
            
    print("Generator training over .. probability real true : {} \n probability fake true : {}".format(prob_real_true(disc_weights), prob_fake_true(gen_weights, disc_weights)))
    # fidelity calculation
    trained_gen(gen_weights)
    rho_gen = dev2.state.reduced_dm(1)
    real_distr()
    rho_real = dev3.state.reduced_dm(1)
    fid = fidelity(rho_gen,rho_real)
    print("fidelity: {}".format(fid))
    f.append(fid)
    if fid>best_f:
        print("New best fid")
        best_fl.append(fid)
        best_dw = disc_weights
        best_gw = gen_weights
        best_f = fid
    
plt.plot(f)
    

In [0]:

trained_gen(best_gw)
rho_gen = dev2.state.reduced_dm(1)
real_distr()
rho_real = dev3.state.reduced_dm(1)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
X, P, W = wigner(rho_gen)
ax.plot_surface(X, P, W, cmap="RdYlGn", lw=0.5, rstride=1, cstride=1)
ax.contour(X, P, W, 10, cmap="RdYlGn", linestyles="solid", offset=-0.17)
ax.set_axis_off()

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
X, P, W = wigner(rho_real)
ax.plot_surface(X, P, W, cmap="RdYlGn", lw=0.5, rstride=1, cstride=1)
ax.contour(X, P, W, 10, cmap="RdYlGn", linestyles="solid", offset=-0.17)
ax.set_axis_off()